# Keyboard Optimizer

Suppose we are tasked with redesigning the computer keyboard. We can take out all of the individual keys and rearrange them however we like.

For example, I'd like to rearrange mine as:

*geortyuiwp\
asdfqhjkl\
zxcvbnm*

How would you do it? How could you make it optimal for you? How could you make it so that when you type you have to travel the shortest distance?

---

To make it a bit easier for us, let's assume that we only use one finger to type. 
If we type the word 'convex', we will have to go 'c' -> 'o' -> 'n' -> 'v' -> 'e' -> 'x', so our individual paths will be 'co', 'on', 'nv', 've' and 'ex'. 



Our problem now becomes: **how do we minimize the average distance the finger would have to travel when typing?**

Let's make this a bit easier for us again: we're going to assume we know what we're going to type and we need to optimize on that.

### Setup

In [ ]:
import string
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

## Loading in the text

In [ ]:
f = open('.txt',"r")

In [ ]:
text = f.read()
text

This is the reading list from the MATH 441 canvas page, but you can replace it with whatever you want!

We need to convert this into counts of pairs of letters, so that we can work out how often our finger needs to move between the keys. Let's start by creating a list of all the words.

In [ ]:
words = []

We import the alphabet - for our use case we are only looking at the 26 letter English alphabet, but we could look at different alphabets or including other characters too.

In [ ]:
alphabet = string.ascii_lowercase


Now we create the matrix which will hold the counts of pairs of letters, or how often our finger needs to move between the letters

In [ ]:
frequency_follower = np.zeros((len(alphabet),len(alphabet)))
# rows are input, columns are output

In [ ]:
for word in words:
    for char_index in range(len(word)-1):
        char_0 = word[char_index]
        char_1 = word[char_index+1]
        if char_0 in alphabet and char_1 in alphabet:
            frequency_follower[alphabet.index(char_0)][alphabet.index(char_1)]+=1


Let's take a look at it:

In [ ]:
np.round(frequency_follower[0:5,0:5])

We normalize it, so that we can calculate the average distance we need to travel per pair.

In [ ]:
frequency_follower/=np.sum(frequency_follower)


## Create the keyboard

We assume the rows are middle aligned, and use this to calculate the distances.

In [ ]:
# first row: 0-9
# second row: 10-18
# third row: 19-25
num_keys = len(alphabet)
key_positions = [[i for i in range(10)],[i for i in range(10,19)],[i for i in range(19,26)]]
row_medians = [np.median(row) for row in key_positions]

keyboard_distances = np.zeros((num_keys,num_keys))
for i in range(num_keys):
    
    i_row = [i in key_row for key_row in key_positions].index(True)
    i_distance_to_middle = row_medians[i_row]-i
    
    for j in range(i,num_keys):
        
        j_row = [j in key_row for key_row in key_positions].index(True)
        j_distance_to_middle = row_medians[j_row]-j
        
        y_distance = abs(i_row-j_row)
        x_distance = abs(i_distance_to_middle-j_distance_to_middle)
        total_distance = np.sqrt(y_distance**2+x_distance**2)
        
        keyboard_distances[i,j] = total_distance
        keyboard_distances[j,i] = total_distance

## Solving the problem

What is the decision variable?\
How can we define our objective function?\
 What about our constraints?

In [ ]:
x = cp.Variable()

In [ ]:
constraints = []
    

In [ ]:
obj = 
problem =

In [ ]:
problem.solve()

What happened here?

## What can we do instead?

What if we started at a known good configuration, and then tried to improve it? Can we use the greedy algorithm/steepest descent?

In [ ]:
initial_configuration = [
    ['q','w','e','r','t','y','u','i','o','p'],
    ['a','s','d','f','g','h','j','k','l'],
    ['z','x','c','v','b','n','m']
]


We can convert that into our decision variable x:

In [ ]:
real_x = np.zeros((len(alphabet),len(alphabet)))
for letter_ind,letter in enumerate(alphabet):
    # Let's find where that letter is and put a 1 in the x matrix at that point
    for row_ind,row in enumerate(key_positions):
        if letter in initial_configuration[row_ind]:
            position = initial_configuration[row_ind].index(letter)
            key = row[position]
            real_x[letter_ind,key]=1
            break

And create a function to calculate the value of the objective function at our x

In [ ]:
calculate_score = 

In [ ]:
calculate_score(real_x)

Let's swap two letters and see how the score compares.

In [ ]:
x = real_x.copy()

letter_0 = alphabet.index('')
letter_1 = alphabet.index('')
key_0 = np.argmax(x[letter_0,:]) # As one value will be 1, the others are 0
key_1 = np.argmax(x[letter_1,:])

x[letter_0,key_0]=0
x[letter_1,key_1]=0
x[letter_0,key_1]=1
x[letter_1,key_0]=1


In [ ]:
calculate_score(x)

Let's also create a function to view the keyboard:

In [ ]:
def print_board(x):
    for row_index,row in enumerate(key_positions):
        row_letters = []
        for key in row:
            row_letters.append(alphabet[np.argmax(x[:,key])])
        if row_index==0:
            print(' '.join(row_letters))
        elif row_index==1:
            print(' '+' '.join(row_letters))
        else:
            print('   '+' '.join(row_letters))

In [ ]:
print_board(real_x)

In [ ]:
print_board(x)

In [ ]:
def swap_letters(x, letter_0, letter_1):
    temp_x = x.copy()
    key_0 = np.argmax(temp_x[letter_0,:])
    key_1 = np.argmax(temp_x[letter_1,:])
    temp_x[letter_0,key_0]=0
    temp_x[letter_1,key_1]=0
    temp_x[letter_0,key_1]=1
    temp_x[letter_1,key_0]=1
    return temp_x

def find_best_improvement(x):
    scores = np.zeros((len(alphabet),len(alphabet))) # This matrix will keep track of the score each swap creates
    for letter_0 in range(len(alphabet)):
        for letter_1 in range(letter_0,len(alphabet)):
            temp_x = swap_letters(x,letter_0,letter_1)
            scores[letter_0,letter_1] = calculate_score(temp_x)
            scores[letter_1,letter_0] = calculate_score(temp_x)
    return  np.unravel_index(np.argmin(scores), scores.shape)

def greedy(starting_x):
    x = starting_x.copy
    for i in range(100): #We're going to do max 100 iterations of the greedy algorithm
        letter_0,letter_1 =find_best_improvement(x) #Find the best swap
        if letter_0==letter_1: # If the best swap is no swap, then the greedy algorithm is finished
            break
        x = swap_letters(x,letter_0,letter_1)
    return x

Let's run our algorithm on the initial keyboard above. 

In [ ]:
final_x = greedy(real_x)

In [ ]:
calculate_score(final_x)

In [ ]:
print_board(final_x)

Have we solved the problem? If not, why not?

## Broadening our approach

As we saw in the TSP, using the greedy algorithm for different starting values will give us different results. This depends on the shape of our solution space.

Let's see what happens with other starting configurations:

In [ ]:
num_random = 

We're going to use tqdm to track our progress- it'll tell us if it's going to take a really long time to finish a loop.

###### Aside: tqdm means 'progress' in Arabic, and is an abbreviation for 'I love you so much' in Spanish - te quiero demasiado

In [ ]:
%pip install tqdm

In [ ]:
from tqdm import tqdm

Now lets create a function to generate random starting positions

In [ ]:
def get_random_configuration():
    x = np.zeros((len(alphabet),len(alphabet)))
    for i in range(len(alphabet)):
        
    return x

Let's generate some results!

In [ ]:
optimal_xs = []
xs_scores = []
for _ in tqdm(range(num_random)):
    x = get_random_configuration()
    x = greedy(x)
    optimal_xs.append(x)
    xs_scores.append(calculate_score(x))
        

We can take a look at the scores of each of the different attempts:

In [ ]:
plt.hist(xs_scores,bins = 20)
plt.xlabel("Average distance")


And let's find the best keyboard that our algorithm found:

In [ ]:
best_x_arg = np.argmin(xs_scores)

In [ ]:
print_board(optimal_xs[best_x_arg])

In [ ]:
xs_scores[best_x_arg]

Have we solved it now?

In [ ]:
len(optimal_xs)

How many different results did we get?

In [ ]:
len(np.unique(optimal_xs,axis=0))

We have found a locally optimal solution, and a pretty good one! But it's not necessarily the best solution.

### Future direction

A couple of potential next steps if you want to take this further:

* How do we include other symbols?
* How do we include other fingers? Can you construct the problem with two fingers?
* How do we solve this problem so that we know we have the best solution? (You might need to use non-python tools, but they're definitely there!)
